# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from IPython.display import display

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
#import CSV
vacation_df = pd.read_csv("weather_data.csv")
vacation_df.head()

,Unnamed: 0,City,Lat,Lng,Country,Date,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Cape Town,-33.9258,18.4232,ZA,1657899713,64.81,76,0,76
1,1,Vanimo,-2.6741,141.3028,PG,1657899713,76.86,91,86,91
2,2,Neuquén,-38.9516,-68.0591,AR,1657899554,39.09,87,75,87
3,3,Avarua,-21.2078,-159.7750,CK,1657899714,73.45,60,75,60
4,4,Beringovskiy,63.0500,179.3167,RU,1657899714,46.22,82,43,82


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
#create dataframe of latitudes and longitudes
locations = vacation_df[["Lat", "Lng"]]

#create list of humidities
humidity = vacation_df["Humidity"]

#create heatmap
fig = gmaps.figure()

#create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=1)

#add the heat layer
fig.add_layer(heat_layer)

#display the heat map
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#create a filtered data frame of ideal conditions and check for null values
filtered_df = vacation_df[(vacation_df['Humidity'] > 70) & (vacation_df['Cloudiness']  < 10) & (vacation_df['Max Temp'] > 60)]
filtered_df.isnull().values.any()

False

In [7]:
#view filtered results
filtered_df

,Unnamed: 0,City,Lat,Lng,Country,Date,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Cape Town,-33.9258,18.4232,ZA,1657899713,64.81,76,0,76
44,44,Port Alfred,-33.5906,26.8910,ZA,1657899728,61.18,81,4,81
50,50,Nouadhibou,20.9310,-17.0347,MR,1657899730,75.18,73,4,73
66,66,Port Elizabeth,-33.9180,25.5701,ZA,1657899735,62.10,81,0,81
81,81,Makakilo City,21.3469,-158.0858,US,1657899740,75.18,74,0,74
154,154,Antalaha,-14.9003,50.2788,MG,1657899768,69.84,84,8,84
167,167,Dandong,40.1292,124.3947,CN,1657899771,67.89,93,2,93
182,182,Ponta do Sol,32.6667,-17.1000,PT,1657899777,75.83,75,3,75
185,185,Nantucket,41.2835,-70.0995,US,1657899778,78.91,72,0,72
221,221,Paphos,34.7667,32.4167,CY,1657899789,85.23,74,0,74


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
#create a dictionary to store search critera
params = {
    "radius": 50000,
    "keyword": "hotel",
    "key": g_key
}

#use the lat/lng we recovered to identify airports
for index, row in filtered_df.iterrows():
    #get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    #change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    #use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params, verify=False)

    # convert to json
    name_address = name_address.json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        filtered_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

/Users/danielmurphy/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps.googleapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/danielmurphy/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/Users/danielmurphy/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



Missing field/result... skipping.


/Users/danielmurphy/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps.googleapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/danielmurphy/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps.googleapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/danielmurphy/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps.googleapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warn

Missing field/result... skipping.


/Users/danielmurphy/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps.googleapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/danielmurphy/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps.googleapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/danielmurphy/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maps.googleapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warn

In [9]:
filtered_df

,Unnamed: 0,City,Lat,Lng,Country,Date,Max Temp,Humidity,Cloudiness,Wind Speed,Hotel Name
0,0,Cape Town,-33.9258,18.4232,ZA,1657899713,64.81,76,0,76,Road Lodge Cape Town International Airport
44,44,Port Alfred,-33.5906,26.8910,ZA,1657899728,61.18,81,4,81,Royal St Andrews Hotel
50,50,Nouadhibou,20.9310,-17.0347,MR,1657899730,75.18,73,4,73,Hotel Delphin-Nouadhibou
66,66,Port Elizabeth,-33.9180,25.5701,ZA,1657899735,62.10,81,0,81,"Radisson Blu Hotel, Port Elizabeth"
81,81,Makakilo City,21.3469,-158.0858,US,1657899740,75.18,74,0,74,Courtyard by Marriott Oahu North Shore
154,154,Antalaha,-14.9003,50.2788,MG,1657899768,69.84,84,8,84,Villa-Malaza
167,167,Dandong,40.1292,124.3947,CN,1657899771,67.89,93,2,93,Hilton Garden Inn Dandong
182,182,Ponta do Sol,32.6667,-17.1000,PT,1657899777,75.83,75,3,75,Calheta Beach - All-inclusive
185,185,Nantucket,41.2835,-70.0995,US,1657899778,78.91,72,0,72,"Holiday Inn Cape Cod-Falmouth, an IHG Hotel"
221,221,Paphos,34.7667,32.4167,CY,1657899789,85.23,74,0,74,St. George Beach Hotel & Spa Resort


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in filtered_df.iterrows()]
locations = filtered_df[["Lat", "Lng"]]

In [15]:
# Add marker layer ontop of heat map
marker_locations = locations
hotel_name = filtered_df['Hotel Name']
heat_locations = vacation_df[["Lat", "Lng"]]

# Display figure
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations, info_box_content=hotel_info)
heat_layer = gmaps.heatmap_layer(heat_locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=1)
fig.add_layer(markers)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))